In [ ]:
%use s2, algoquant
val linearShrinkageMeanEstimatorParam = 0.5
val linearShrinkageCovEstimatorParam = 0.5
val riskAverse = 1.0
val csvdata = CsvCacheProvider("sample_assets.csv", Currencies.USD)
val simInterval = csvdata.dataInterval()
val rebPer = Period.parse("P1M")
val timeWin = Period.parse("P6M")
val sim = OptimizationAlgorithmComparisonSimulation(
    csvdata.allProducts(simInterval),
    csvdata,
    csvdata,
    rebPer,
    timeWin
)
// long only constraints
val stocksCache = BufferCache<List<String>>()
val stockTics = mutableListOf<String>()
csvdata.allProducts(csvdata.dataInterval()).forEach { p ->
    stockTics.add(p.symbol())
}
stocksCache.add(csvdata.dataInterval().start(),stockTics)
val constraint = MarkowitzLongOnlyConstraint().addLongOnlyPositionLimit(stocksCache)
// optimizer
val optimAlgo = MarkowitzDynamicConstraintOptimizationAlgorithm(riskAverse, constraint)
// estimators
optimAlgo.meanEstimator = MeanAnnualizingWrapper(LinearShrinkageMeanEstimator(linearShrinkageMeanEstimatorParam), rebPer)
optimAlgo.covarianceEstimator = CovarianceAnnualizingWrapper(LinearShrinkageCovarianceEstimator(linearShrinkageCovEstimatorParam), rebPer)
// run simulation
sim.addAlgorithm("simulation", optimAlgo)
sim.setRiskFreeRateLookup(DbRiskFreeRateLookup().newRiskFreeRateLookup(simInterval))
sim.setBenchmarkLookupFactory("SP500", BenchmarkLookupFactory())

val result = sim.run(simInterval)
println("success")